# सत्र 4 – SLM बनाम LLM तुलना

एक छोटे भाषा मॉडल और Foundry Local के माध्यम से चलने वाले बड़े मॉडल के बीच विलंबता और नमूना प्रतिक्रिया गुणवत्ता की तुलना करें।


## ⚡ त्वरित शुरुआत

**मेमोरी-अनुकूलित सेटअप (अपडेटेड):**
1. मॉडल्स स्वचालित रूप से CPU वेरिएंट चुनते हैं (किसी भी हार्डवेयर पर काम करता है)
2. `qwen2.5-3b` का उपयोग करता है बजाय 7B के (लगभग 4GB RAM बचाता है)
3. पोर्ट का स्वचालित पता लगाना (कोई मैनुअल कॉन्फ़िगरेशन नहीं)
4. कुल आवश्यक RAM: ~8GB अनुशंसित (मॉडल्स + OS)

**टर्मिनल सेटअप (30 सेकंड):**
```bash
foundry service start
foundry model run phi-4-mini
foundry model run qwen2.5-3b
```

इसके बाद इस नोटबुक को चलाएं! 🚀


### व्याख्या: निर्भरता स्थापना
न्यूनतम पैकेज (`foundry-local-sdk`, `openai`, `numpy`) स्थापित करता है जो समय निर्धारण और चैट अनुरोधों के लिए आवश्यक हैं। इसे सुरक्षित रूप से बार-बार चलाया जा सकता है।


# परिदृश्य
एक छोटे भाषा मॉडल (SLM) की तुलना एक बड़े मॉडल से एक ही प्रॉम्प्ट पर करें ताकि निम्नलिखित का विश्लेषण किया जा सके:
- **प्रतिक्रिया समय का अंतर** (दीवार घड़ी सेकंड में)
- **टोकन उपयोग** (यदि उपलब्ध हो) थ्रूपुट के लिए एक संकेतक के रूप में
- **गुणात्मक आउटपुट का नमूना** त्वरित मूल्यांकन के लिए
- **गति वृद्धि की गणना** प्रदर्शन लाभ को मापने के लिए

**पर्यावरणीय वेरिएबल्स:**
- `SLM_ALIAS` - छोटा भाषा मॉडल (डिफ़ॉल्ट: phi-4-mini, ~4GB RAM)
- `LLM_ALIAS` - बड़ा भाषा मॉडल (डिफ़ॉल्ट: qwen2.5-7b, ~7GB RAM)
- `COMPARE_PROMPT` - तुलना के लिए परीक्षण प्रॉम्प्ट
- `COMPARE_RETRIES` - लचीलापन के लिए पुनः प्रयास (डिफ़ॉल्ट: 2)
- `FOUNDRY_LOCAL_ENDPOINT` - सेवा एंडपॉइंट को ओवरराइड करें (यदि सेट नहीं है तो स्वचालित रूप से पता लगाया जाएगा)

**यह कैसे काम करता है (आधिकारिक SDK पैटर्न):**
1. **FoundryLocalManager** Foundry Local सेवा को प्रारंभ और प्रबंधित करता है
2. यदि सेवा चालू नहीं है तो यह स्वचालित रूप से शुरू हो जाती है (कोई मैनुअल सेटअप आवश्यक नहीं)
3. मॉडल को उपनामों से स्वचालित रूप से ठोस आईडी में हल किया जाता है
4. हार्डवेयर-अनुकूलित वेरिएंट का चयन किया जाता है (CUDA, NPU, या CPU)
5. OpenAI-संगत क्लाइंट चैट पूर्णता करता है
6. मेट्रिक्स कैप्चर किए जाते हैं: प्रतिक्रिया समय, टोकन, आउटपुट गुणवत्ता
7. परिणामों की तुलना करके गति वृद्धि अनुपात की गणना की जाती है

यह सूक्ष्म तुलना यह तय करने में मदद करती है कि आपके उपयोग के मामले के लिए बड़े मॉडल की ओर रूटिंग कब उचित है।

**SDK संदर्भ:** 
- Python SDK: https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local
- वर्कशॉप यूटिल्स: ../samples/workshop_utils.py से आधिकारिक पैटर्न का उपयोग करता है

**मुख्य लाभ:**
- ✅ स्वचालित सेवा खोज और प्रारंभिककरण
- ✅ यदि सेवा चालू नहीं है तो स्वचालित रूप से शुरू होती है
- ✅ अंतर्निहित मॉडल समाधान और कैशिंग
- ✅ हार्डवेयर अनुकूलन (CUDA/NPU/CPU)
- ✅ OpenAI SDK संगतता
- ✅ पुनः प्रयास के साथ मजबूत त्रुटि प्रबंधन
- ✅ स्थानीय अनुमान (कोई क्लाउड API आवश्यक नहीं)


## 🚨 आवश्यक शर्तें: Foundry Local चालू होना चाहिए!

**इस नोटबुक को चलाने से पहले**, सुनिश्चित करें कि Foundry Local सेवा सेटअप हो चुकी है:

### त्वरित प्रारंभ कमांड (टर्मिनल में चलाएं):

```bash
# 1. Start the Foundry Local service
foundry service start

# 2. Load the default models used in this comparison (CPU-optimized)
foundry model run phi-4-mini
foundry model run qwen2.5-3b

# 3. Verify models are loaded
foundry model ls

# 4. Check service health
foundry service status
```

### वैकल्पिक मॉडल (यदि डिफ़ॉल्ट उपलब्ध नहीं हैं):

```bash
# Even smaller alternatives (if memory is very limited)
foundry model run phi-3.5-mini
foundry model run qwen2.5-0.5b

# Or update the environment variables in this notebook:
# SLM_ALIAS = 'phi-3.5-mini'
# LLM_ALIAS = 'qwen2.5-1.5b'  # Or qwen2.5-0.5b for minimum memory
```

⚠️ **यदि आप इन चरणों को छोड़ते हैं**, तो नीचे नोटबुक सेल चलाने पर आपको `APIConnectionError` दिखाई देगा।


In [29]:
# Install dependencies
!pip install -q foundry-local-sdk openai numpy requests

### व्याख्या: कोर आयात
समय उपयोगिताओं और Foundry Local / OpenAI क्लाइंट्स को शामिल करता है, जो मॉडल जानकारी प्राप्त करने और चैट पूर्णता करने के लिए उपयोग किए जाते हैं।


In [30]:
import os, time, json
from foundry_local import FoundryLocalManager
from openai import OpenAI
import sys
sys.path.append('../samples')
from workshop_utils import get_client, chat_once

### व्याख्या: उपनाम और प्रॉम्प्ट सेटअप  
छोटे और बड़े मॉडल के लिए पर्यावरण-कॉन्फ़िगर करने योग्य उपनामों को परिभाषित करता है, साथ ही तुलना प्रॉम्प्ट भी। विभिन्न मॉडल परिवारों या कार्यों के साथ प्रयोग करने के लिए पर्यावरण वेरिएबल्स को समायोजित करें।


In [31]:
# Default to CPU models for better memory efficiency
SLM = os.getenv('SLM_ALIAS', 'phi-4-mini')  # Auto-selects CPU variant
LLM = os.getenv('LLM_ALIAS', 'qwen2.5-3b')  # Smaller LLM, more memory-friendly
PROMPT = os.getenv('COMPARE_PROMPT', 'List 5 benefits of local AI inference.')
# Endpoint is now managed by FoundryLocalManager - it auto-detects or can be overridden
ENDPOINT = os.getenv('FOUNDRY_LOCAL_ENDPOINT', None)

### 💡 मेमोरी-अनुकूलित कॉन्फ़िगरेशन

**यह नोटबुक डिफ़ॉल्ट रूप से मेमोरी-कुशल मॉडल का उपयोग करता है:**
- `phi-4-mini` → ~4GB RAM (Foundry Local स्वचालित रूप से CPU संस्करण चुनता है)
- `qwen2.5-3b` → ~3GB RAM (7B के बजाय जो ~7GB+ की आवश्यकता होती है)

**पोर्ट ऑटो-डिटेक्शन:**
- Foundry Local विभिन्न पोर्ट का उपयोग कर सकता है (आमतौर पर 55769 या 59959)
- नीचे दिया गया डायग्नोस्टिक सेल सही पोर्ट को स्वचालित रूप से पहचानता है
- कोई मैनुअल कॉन्फ़िगरेशन की आवश्यकता नहीं है!

**यदि आपके पास सीमित RAM (<8GB) है, तो और भी छोटे मॉडल का उपयोग करें:**
```python
SLM = 'phi-3.5-mini'      # ~2GB
LLM = 'qwen2.5-0.5b'      # ~500MB
```


In [32]:
# Display current configuration
print("="*60)
print("CURRENT CONFIGURATION")
print("="*60)
print(f"SLM Model:     {SLM}")
print(f"LLM Model:     {LLM}")
print(f"SDK Pattern:   FoundryLocalManager (official)")
print(f"Endpoint:      {ENDPOINT or 'Auto-detect'}")
print(f"Test Prompt:   {PROMPT[:50]}...")
print(f"Retry Count:   2")
print("="*60)
print("\n💡 Using official Foundry SDK pattern from workshop_utils")
print("   → FoundryLocalManager handles service lifecycle")
print("   → Automatic model resolution and hardware optimization")
print("   → OpenAI-compatible API for inference")

CURRENT CONFIGURATION
SLM Model:     phi-4-mini
LLM Model:     qwen2.5-7b
SDK Pattern:   FoundryLocalManager (official)
Endpoint:      Auto-detect
Test Prompt:   List 5 benefits of local AI inference....
Retry Count:   2

💡 Using official Foundry SDK pattern from workshop_utils
   → FoundryLocalManager handles service lifecycle
   → Automatic model resolution and hardware optimization
   → OpenAI-compatible API for inference


### व्याख्या: निष्पादन सहायक (Foundry SDK पैटर्न)
यह आधिकारिक Foundry Local SDK पैटर्न का उपयोग करता है, जैसा कि Workshop नमूनों में दस्तावेज़ित किया गया है:

**दृष्टिकोण:**
- **FoundryLocalManager** - Foundry Local सेवा को प्रारंभ और प्रबंधित करता है
- **स्वचालित पहचान** - स्वतः ही एंडपॉइंट का पता लगाता है और सेवा जीवनचक्र को संभालता है
- **मॉडल समाधान** - उपनामों को पूर्ण मॉडल आईडी में बदलता है (जैसे, phi-4-mini → phi-4-mini-instruct-cpu)
- **हार्डवेयर अनुकूलन** - उपलब्ध हार्डवेयर के लिए सबसे अच्छा संस्करण चुनता है (CUDA, NPU, या CPU)
- **OpenAI क्लाइंट** - OpenAI-संगत API एक्सेस के लिए मैनेजर के एंडपॉइंट के साथ कॉन्फ़िगर किया गया

**लचीलापन विशेषताएँ:**
- एक्सपोनेंशियल बैकऑफ रीट्राई लॉजिक (पर्यावरण के माध्यम से कॉन्फ़िगर करने योग्य)
- यदि सेवा चालू नहीं है तो स्वचालित रूप से सेवा प्रारंभ
- प्रारंभिककरण के बाद कनेक्शन सत्यापन
- विस्तृत त्रुटि रिपोर्टिंग के साथ सहज त्रुटि प्रबंधन
- बार-बार प्रारंभिककरण से बचने के लिए मॉडल कैशिंग

**परिणाम संरचना:**
- विलंबता मापन (दीवार घड़ी समय)
- टोकन उपयोग ट्रैकिंग (यदि उपलब्ध हो)
- नमूना आउटपुट (पढ़ने में आसानी के लिए छोटा किया गया)
- असफल अनुरोधों के लिए त्रुटि विवरण

यह पैटर्न workshop_utils मॉड्यूल का उपयोग करता है, जो आधिकारिक SDK पैटर्न का अनुसरण करता है।

**SDK संदर्भ:**
- मुख्य रिपॉजिटरी: https://github.com/microsoft/Foundry-Local
- Python SDK: https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local
- Workshop Utils: ../samples/workshop_utils.py


In [39]:
def setup(alias: str, endpoint: str = None, retries: int = 3):
    """
    Initialize a Foundry Local model connection using official SDK pattern.
    
    This follows the workshop_utils pattern which uses FoundryLocalManager
    to properly initialize the Foundry Local service and resolve models.
    
    Args:
        alias: Model alias (e.g., 'phi-4-mini', 'qwen2.5-3b')
        endpoint: Optional endpoint override (usually auto-detected)
        retries: Number of connection attempts (default: 3)
    
    Returns:
        tuple: (manager, client, model_id, metadata) or (None, None, alias, error_metadata) if failed
    """
    import time
    
    last_err = None
    current_delay = 2  # seconds
    
    for attempt in range(1, retries + 1):
        try:
            print(f"[Init] Connecting to '{alias}' (attempt {attempt}/{retries})...")
            
            # Use the workshop utility which follows the official SDK pattern
            manager, client, model_id = get_client(alias, endpoint=endpoint)
            
            print(f"[OK] Connected to '{alias}' -> {model_id}")
            print(f"     Endpoint: {manager.endpoint}")
            
            return manager, client, model_id, {
                'endpoint': manager.endpoint,
                'resolved': model_id,
                'attempts': attempt,
                'status': 'success'
            }
            
        except Exception as e:
            last_err = e
            error_msg = str(e)
            
            # Provide helpful error messages
            if "Connection error" in error_msg or "connection refused" in error_msg.lower():
                print(f"[ERROR] Cannot connect to Foundry Local service")
                print(f"        → Is the service running? Try: foundry service start")
                print(f"        → Is the model loaded? Try: foundry model run {alias}")
            elif "not found" in error_msg.lower():
                print(f"[ERROR] Model '{alias}' not found in catalog")
                print(f"        → Available models: Run 'foundry model ls' in terminal")
                print(f"        → Download model: Run 'foundry model download {alias}'")
            else:
                print(f"[ERROR] Setup failed: {type(e).__name__}: {error_msg}")
            
            if attempt < retries:
                print(f"[Retry] Waiting {current_delay:.1f}s before retry...")
                time.sleep(current_delay)
                current_delay *= 2  # Exponential backoff
    
    # All retries failed - provide actionable guidance
    print(f"\n❌ Failed to initialize '{alias}' after {retries} attempts")
    print(f"   Last error: {type(last_err).__name__}: {str(last_err)}")
    print(f"\n💡 Troubleshooting steps:")
    print(f"   1. Ensure Foundry Local service is running:")
    print(f"      → foundry service status")
    print(f"      → foundry service start (if not running)")
    print(f"   2. Ensure model is loaded:")
    print(f"      → foundry model run {alias}")
    print(f"   3. Check available models:")
    print(f"      → foundry model ls")
    print(f"   4. Try alternative models if '{alias}' isn't available")
    
    return None, None, alias, {
        'error': f"{type(last_err).__name__}: {str(last_err)}",
        'endpoint': endpoint or 'auto-detect',
        'attempts': retries,
        'status': 'failed'
    }


def run(client, model_id: str, prompt: str, max_tokens: int = 180, temperature: float = 0.5):
    """
    Run inference with the configured model using OpenAI SDK.
    
    Args:
        client: OpenAI client instance (configured for Foundry Local)
        model_id: Model identifier (resolved from alias)
        prompt: Input prompt
        max_tokens: Maximum response tokens (default: 180)
        temperature: Sampling temperature (default: 0.5)
    
    Returns:
        dict: Response with timing, tokens, and content
    """
    import time
    
    start = time.time()
    
    try:
        response = client.chat.completions.create(
            model=model_id,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=temperature
        )
        
        elapsed = time.time() - start
        
        # Extract response details
        content = response.choices[0].message.content
        
        # Try to extract token usage from multiple possible locations
        usage_info = {}
        if hasattr(response, 'usage') and response.usage:
            usage_info['prompt_tokens'] = getattr(response.usage, 'prompt_tokens', None)
            usage_info['completion_tokens'] = getattr(response.usage, 'completion_tokens', None)
            usage_info['total_tokens'] = getattr(response.usage, 'total_tokens', None)
        
        # Calculate approximate token count if API doesn't provide it
        # Rough estimate: ~4 characters per token for English text
        if not usage_info.get('total_tokens'):
            estimated_prompt_tokens = len(prompt) // 4
            estimated_completion_tokens = len(content) // 4
            estimated_total = estimated_prompt_tokens + estimated_completion_tokens
            usage_info['estimated_tokens'] = estimated_total
            usage_info['estimated_prompt_tokens'] = estimated_prompt_tokens
            usage_info['estimated_completion_tokens'] = estimated_completion_tokens
        
        return {
            'status': 'success',
            'content': content,
            'elapsed_sec': elapsed,
            'tokens': usage_info.get('total_tokens') or usage_info.get('estimated_tokens'),
            'usage': usage_info,
            'model': model_id
        }
        
    except Exception as e:
        elapsed = time.time() - start
        return {
            'status': 'error',
            'error': f"{type(e).__name__}: {str(e)}",
            'elapsed_sec': elapsed,
            'model': model_id
        }


print("✅ Execution helpers defined: setup(), run()")
print("   → Uses workshop_utils for proper SDK integration")
print("   → setup() initializes with FoundryLocalManager")
print("   → run() executes inference via OpenAI-compatible API")
print("   → Token counting: Uses API data or estimates if unavailable")

✅ Execution helpers defined: setup(), run()
   → Uses workshop_utils for proper SDK integration
   → setup() initializes with FoundryLocalManager
   → run() executes inference via OpenAI-compatible API
   → Token counting: Uses API data or estimates if unavailable


### व्याख्या: प्री-फ्लाइट सेल्फ-टेस्ट
FoundryLocalManager का उपयोग करके दोनों मॉडलों के लिए एक हल्का कनेक्टिविटी चेक चलाता है। यह सुनिश्चित करता है:
- सेवा सुलभ है
- मॉडल प्रारंभ किए जा सकते हैं
- उपनाम वास्तविक मॉडल आईडी में बदलते हैं
- तुलना चलाने से पहले कनेक्शन स्थिर है

setup() फ़ंक्शन workshop_utils से आधिकारिक SDK पैटर्न का उपयोग करता है।


In [34]:
# Simplified diagnostic: Just verify service is accessible
import requests

def check_foundry_service():
    """Quick diagnostic to verify Foundry Local is running."""
    # Try common ports
    endpoints_to_try = [
        "http://localhost:59959",
        "http://127.0.0.1:59959", 
        "http://localhost:55769",
        "http://127.0.0.1:55769",
    ]
    
    print("[Diagnostic] Checking Foundry Local service...")
    
    for endpoint in endpoints_to_try:
        try:
            response = requests.get(f"{endpoint}/health", timeout=2)
            if response.status_code == 200:
                print(f"✅ Service is running at {endpoint}")
                
                # Try to list models
                try:
                    models_response = requests.get(f"{endpoint}/v1/models", timeout=2)
                    if models_response.status_code == 200:
                        models_data = models_response.json()
                        model_count = len(models_data.get('data', []))
                        print(f"✅ Found {model_count} models available")
                        if model_count > 0:
                            print("   Models:", [m.get('id', 'unknown') for m in models_data.get('data', [])[:5]])
                except Exception as e:
                    print(f"⚠️  Could not list models: {e}")
                
                return endpoint
        except requests.exceptions.ConnectionError:
            continue
        except Exception as e:
            print(f"⚠️  Error checking {endpoint}: {e}")
    
    print("\n❌ Foundry Local service not found!")
    print("\n💡 To fix this:")
    print("   1. Open a terminal")
    print("   2. Run: foundry service start")
    print("   3. Run: foundry model run phi-4-mini")
    print("   4. Run: foundry model run qwen2.5-3b")
    print("   5. Re-run this notebook")
    return None

# Run diagnostic
discovered_endpoint = check_foundry_service()

if discovered_endpoint:
    print(f"\n✅ Service detected (will be managed by FoundryLocalManager)")
else:
    print(f"\n⚠️  No service detected - FoundryLocalManager will attempt to start it")

[Diagnostic] Checking Foundry Local service...

❌ Foundry Local service not found!

💡 To fix this:
   1. Open a terminal
   2. Run: foundry service start
   3. Run: foundry model run phi-4-mini
   4. Run: foundry model run qwen2.5-3b
   5. Re-run this notebook

⚠️  No service detected - FoundryLocalManager will attempt to start it


In [35]:
# Quick Fix: Start service and load models from notebook
# Uncomment the commands you need:

# !foundry service start
# !foundry model run phi-4-mini
# !foundry model run qwen2.5-3b
# !foundry model ls

print("⚠️  The commands above are commented out.")
print("Uncomment them if you want to start the service from the notebook.")
print("")
print("💡 Recommended: Run these commands in a separate terminal instead:")
print("   foundry service start")
print("   foundry model run phi-4-mini")
print("   foundry model run qwen2.5-3b")

⚠️  The commands above are commented out.
Uncomment them if you want to start the service from the notebook.

💡 Recommended: Run these commands in a separate terminal instead:
   foundry service start
   foundry model run phi-4-mini
   foundry model run qwen2.5-3b


### 🛠️ त्वरित समाधान: नोटबुक से फाउंड्री लोकल शुरू करें (वैकल्पिक)

यदि ऊपर के डायग्नोस्टिक से पता चलता है कि सेवा चालू नहीं है, तो आप इसे यहां से शुरू करने की कोशिश कर सकते हैं:

**नोट:** यह विंडोज़ पर सबसे अच्छा काम करता है। अन्य प्लेटफार्मों पर, टर्मिनल कमांड का उपयोग करें।


### ⚠️ कनेक्शन त्रुटियों का समाधान

यदि आप `APIConnectionError` देख रहे हैं, तो हो सकता है कि Foundry Local सेवा चालू न हो या मॉडल लोड न किए गए हों। इन चरणों को आज़माएं:

**1. सेवा की स्थिति जांचें:**
```bash
# In a terminal (not in notebook):
foundry service status
```

**2. सेवा शुरू करें (यदि चालू नहीं है):**
```bash
foundry service start
```

**3. आवश्यक मॉडल लोड करें:**
```bash
# Load the models needed for comparison
foundry model run phi-4-mini
foundry model run qwen2.5-7b

# Or use alternative models:
foundry model run phi-3.5-mini
foundry model run qwen2.5-3b
```

**4. सुनिश्चित करें कि मॉडल उपलब्ध हैं:**
```bash
foundry model ls
```

**सामान्य समस्याएं:**
- ❌ सेवा चालू नहीं है → `foundry service start` चलाएं
- ❌ मॉडल लोड नहीं किए गए हैं → `foundry model run <model-name>` चलाएं
- ❌ पोर्ट संघर्ष → जांचें कि कोई अन्य सेवा पोर्ट का उपयोग कर रही है
- ❌ फ़ायरवॉल ब्लॉक कर रहा है → सुनिश्चित करें कि स्थानीय कनेक्शन की अनुमति है

**त्वरित समाधान:** प्री-फ्लाइट चेक से पहले नीचे दिया गया डायग्नोस्टिक सेल चलाएं।


In [36]:
preflight = {}
retries = 2  # Number of retry attempts

for a in (SLM, LLM):
    mgr, c, mid, info = setup(a, endpoint=ENDPOINT, retries=retries)
    # Keep the original status from info (either 'success' or 'failed')
    preflight[a] = info

print('\n[Pre-flight Check]')
for alias, details in preflight.items():
    status_icon = '✅' if details['status'] == 'success' else '❌'
    print(f"  {status_icon} {alias}: {details['status']} - {details.get('resolved', details.get('error', 'unknown'))}")

preflight

[Init] Connecting to 'phi-4-mini' (attempt 1/2)...
[OK] Connected to 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1

[Pre-flight Check]
  ✅ phi-4-mini: success - Phi-4-mini-instruct-cuda-gpu:4
  ✅ qwen2.5-7b: success - qwen2.5-7b-instruct-cuda-gpu:3


{'phi-4-mini': {'endpoint': 'http://127.0.0.1:59959/v1',
  'resolved': 'Phi-4-mini-instruct-cuda-gpu:4',
  'attempts': 1,
  'status': 'success'},
 'qwen2.5-7b': {'endpoint': 'http://127.0.0.1:59959/v1',
  'resolved': 'qwen2.5-7b-instruct-cuda-gpu:3',
  'attempts': 1,
  'status': 'success'}}

### ✅ प्री-फ्लाइट चेक: मॉडल उपलब्धता

यह सेल तुलना शुरू करने से पहले यह सुनिश्चित करता है कि दोनों मॉडल कॉन्फ़िगर किए गए एंडपॉइंट पर उपलब्ध हैं।


### व्याख्या: रन तुलना और परिणाम संग्रह करें
आधिकारिक Foundry SDK पैटर्न का उपयोग करते हुए दोनों उपनामों पर पुनरावृत्ति करता है:
1. प्रत्येक मॉडल को setup() के साथ प्रारंभ करें (FoundryLocalManager का उपयोग करता है)
2. OpenAI-संगत API के साथ अनुमान लगाएं
3. विलंबता, टोकन और नमूना आउटपुट कैप्चर करें
4. तुलनात्मक विश्लेषण के साथ JSON सारांश तैयार करें

यह session04/model_compare.py में वर्कशॉप नमूनों के समान पैटर्न का अनुसरण करता है।


In [40]:
results = []
retries = 2  # Number of retry attempts

for alias in (SLM, LLM):
    mgr, client, mid, info = setup(alias, endpoint=ENDPOINT, retries=retries)
    if client:
        r = run(client, mid, PROMPT)
        results.append({'alias': alias, **r})
    else:
        # If setup failed, record error
        results.append({
            'alias': alias,
            'status': 'error',
            'error': info.get('error', 'Setup failed'),
            'elapsed_sec': 0,
            'tokens': None,
            'model': alias
        })

# Display results
print(json.dumps(results, indent=2))

# Quick comparative view
print('\n' + '='*80)
print('COMPARISON SUMMARY')
print('='*80)
print(f"{'Alias':<20} {'Status':<15} {'Latency(s)':<15} {'Tokens':<15}")
print('-'*80)

for row in results:
    status = row.get('status', 'unknown')
    status_icon = '✅' if status == 'success' else '❌'
    latency_str = f"{row.get('elapsed_sec', 0):.3f}" if row.get('elapsed_sec') else 'N/A'
    
    # Handle token display - show if available or indicate estimated
    tokens = row.get('tokens')
    usage = row.get('usage', {})
    if tokens:
        if 'estimated_tokens' in usage:
            tokens_str = f"~{tokens} (est.)"
        else:
            tokens_str = str(tokens)
    else:
        tokens_str = 'N/A'
    
    print(f"{status_icon} {row['alias']:<18} {status:<15} {latency_str:<15} {tokens_str:<15}")

print('-'*80)

# Show detailed token breakdown if available
print("\nDetailed Token Usage:")
for row in results:
    if row.get('status') == 'success' and row.get('usage'):
        usage = row['usage']
        print(f"\n  {row['alias']}:")
        if 'prompt_tokens' in usage and usage['prompt_tokens']:
            print(f"    Prompt tokens:     {usage['prompt_tokens']}")
            print(f"    Completion tokens: {usage['completion_tokens']}")
            print(f"    Total tokens:      {usage['total_tokens']}")
        elif 'estimated_tokens' in usage:
            print(f"    Estimated prompt:     {usage['estimated_prompt_tokens']}")
            print(f"    Estimated completion: {usage['estimated_completion_tokens']}")
            print(f"    Estimated total:      {usage['estimated_tokens']}")
            print(f"    (API did not provide token counts - using ~4 chars/token estimate)")

print('\n' + '='*80)

# Calculate speedup if both succeeded
if len(results) == 2 and all(r.get('status') == 'success' and r.get('elapsed_sec') for r in results):
    speedup = results[1]['elapsed_sec'] / results[0]['elapsed_sec']
    print(f"\n💡 SLM is {speedup:.2f}x faster than LLM for this prompt")
    
    # Compare token throughput if available
    slm_tokens = results[0].get('tokens', 0)
    llm_tokens = results[1].get('tokens', 0)
    if slm_tokens and llm_tokens:
        slm_tps = slm_tokens / results[0]['elapsed_sec']
        llm_tps = llm_tokens / results[1]['elapsed_sec']
        print(f"   SLM throughput: {slm_tps:.1f} tokens/sec")
        print(f"   LLM throughput: {llm_tps:.1f} tokens/sec")
        
elif any(r.get('status') == 'error' for r in results):
    print(f"\n⚠️  Some models failed - check errors above")
    print("   Ensure Foundry Local is running: foundry service start")
    print("   Ensure models are loaded: foundry model run <model-name>")

results

[Init] Connecting to 'phi-4-mini' (attempt 1/2)...
[OK] Connected to 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1
[
  {
    "alias": "phi-4-mini",
    "status": "success",
    "content": "1. Reduced Latency: Local AI inference can significantly reduce latency by processing data closer to the source, which is particularly beneficial for real-time applications such as autonomous vehicles or augmented reality.\n\n2. Enhanced Privacy: By keeping data processing local, sensitive information is less likely to be exposed to external networks, thereby enhancing privacy and security.\n\n3. Lower Bandwidth Usage: Local AI inference reduces the n

[{'alias': 'phi-4-mini',
  'status': 'success',
  'content': '1. Reduced Latency: Local AI inference can significantly reduce latency by processing data closer to the source, which is particularly beneficial for real-time applications such as autonomous vehicles or augmented reality.\n\n2. Enhanced Privacy: By keeping data processing local, sensitive information is less likely to be exposed to external networks, thereby enhancing privacy and security.\n\n3. Lower Bandwidth Usage: Local AI inference reduces the need for data transmission over the network, which can save bandwidth and reduce the risk of network congestion.\n\n4. Improved Reliability: Local processing can be more reliable, as it is less dependent on network connectivity. This is particularly important in scenarios where network connectivity is unreliable or intermittent.\n\n5. Scalability: Local AI inference can be easily scaled by adding more local processing units, making it easier to handle increasing data volumes or m

### परिणामों की व्याख्या

**मुख्य मापदंड:**
- **लेटेंसी**: कम होना बेहतर है - तेज प्रतिक्रिया समय को दर्शाता है
- **टोकन**: अधिक थ्रूपुट = अधिक टोकन संसाधित किए गए
- **रूट**: पुष्टि करता है कि कौन सा API एंडपॉइंट उपयोग किया गया

**SLM बनाम LLM कब उपयोग करें:**
- **SLM (छोटा भाषा मॉडल)**: तेज प्रतिक्रियाएं, कम संसाधन उपयोग, सरल कार्यों के लिए उपयुक्त
- **LLM (बड़ा भाषा मॉडल)**: उच्च गुणवत्ता, बेहतर तर्क क्षमता, जब गुणवत्ता सबसे महत्वपूर्ण हो तब उपयोग करें

**अगले कदम:**
1. विभिन्न प्रॉम्प्ट आज़माएं और देखें कि जटिलता तुलना को कैसे प्रभावित करती है
2. अन्य मॉडल जोड़ों के साथ प्रयोग करें
3. कार्य की जटिलता के आधार पर बुद्धिमानी से रूट करने के लिए वर्कशॉप राउटर सैंपल (सेशन 06) का उपयोग करें


In [38]:
# Final Validation Check
print("="*70)
print("VALIDATION SUMMARY")
print("="*70)
print(f"✅ SLM Model: {SLM}")
print(f"✅ LLM Model: {LLM}")
print(f"✅ Using Foundry SDK Pattern: workshop_utils with FoundryLocalManager")
print(f"✅ Pre-flight passed: {all(v['status'] == 'success' for v in preflight.values()) if 'preflight' in dir() else 'Not run yet'}")
print(f"✅ Comparison completed: {len(results) == 2 if 'results' in dir() else 'Not run yet'}")
print(f"✅ Both models responded: {all(r.get('status') == 'success' for r in results) if 'results' in dir() and results else 'Not run yet'}")
print("="*70)

# Check for common configuration issues
issues = []
if 'LLM' in dir() and LLM not in ['qwen2.5-3b', 'qwen2.5-0.5b', 'qwen2.5-1.5b', 'qwen2.5-7b', 'phi-3.5-mini']:
    issues.append(f"⚠️  LLM is '{LLM}' - expected qwen2.5-3b for memory efficiency")
if 'preflight' in dir() and not all(v['status'] == 'success' for v in preflight.values()):
    issues.append("⚠️  Pre-flight check failed - models not accessible")
if 'results' in dir() and results and not all(r.get('status') == 'success' for r in results):
    issues.append("⚠️  Comparison incomplete - check for errors above")

if not issues and 'results' in dir() and results and all(r.get('status') == 'success' for r in results):
    print("🎉 ALL CHECKS PASSED! Notebook completed successfully.")
    print(f"   SLM ({SLM}) vs LLM ({LLM}) comparison completed.")
    if len(results) == 2:
        speedup = results[1]['elapsed_sec'] / results[0]['elapsed_sec'] if results[0]['elapsed_sec'] > 0 else 0
        print(f"   Performance: SLM is {speedup:.2f}x faster")
elif issues:
    print("\n⚠️  Issues detected:")
    for issue in issues:
        print(f"   {issue}")
    print("\n💡 Troubleshooting:")
    print("   1. Ensure service is running: foundry service start")
    print("   2. Load models: foundry model run phi-4-mini && foundry model run qwen2.5-7b")
    print("   3. Check model list: foundry model ls")
else:
    print("\n💡 Run all cells above first, then re-run this validation.")
print("="*70)

VALIDATION SUMMARY
✅ SLM Model: phi-4-mini
✅ LLM Model: qwen2.5-7b
✅ Using Foundry SDK Pattern: workshop_utils with FoundryLocalManager
✅ Pre-flight passed: True
✅ Comparison completed: True
✅ Both models responded: True
🎉 ALL CHECKS PASSED! Notebook completed successfully.
   SLM (phi-4-mini) vs LLM (qwen2.5-7b) comparison completed.
   Performance: SLM is 5.14x faster



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता के लिए प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को आधिकारिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
